In [ ]:
import talib
import pandas as pd
import numpy as np
from openapi_client import openapi
import json as js
import datetime
from pandas_datareader import data as pdr
import yahoo_fin.stock_info as si
from alpha_vantage.sectorperformance import SectorPerformances
import configparser  # импортируем библиотеку



In [ ]:
config = configparser.ConfigParser()
config['DEFAULT'] = {'token': '',
                    'path_to_page': '',
                    'path_to_excel': ''}

with open('', 'w') as configfile:
    config.write(configfile)

In [ ]:
config['DEFAULT']['token']

In [ ]:
config_2 = configparser.ConfigParser()

In [ ]:
config_2.read("")

In [ ]:
config_2['DEFAULT']['token']

In [ ]:
config = configparser.ConfigParser()  # создаём объекта парсера
config.read("")  # читаем конфиг

In [ ]:
datetime.datetime.now().strftime("%Y-%m-%d %H:%M")

In [ ]:
final_stocks_path_in = ''

with open(final_stocks_path_in) as pt:
    total_tickers = pd.read_csv(pt,sep=';',encoding='utf8')

In [ ]:
tinkoff = {}
list_of_stocs=list(total_tickers['ticker'][0:588])
for ticker in list_of_stocs:
    tinkoff[ticker] = si.get_data(ticker, start_date='25-10-2021', index_as_date = False, interval='1d').sort_values(by='date')

data = pd.concat(tinkoff)
data.reset_index(drop=True, inplace=True)

In [ ]:
data.head(2)

In [ ]:
datetime.datetime.today().strftime('%d-%m-%Y')

In [ ]:
(datetime.datetime.today() - datetime.timedelta(days=15)).strftime('%d-%m-%Y')

In [ ]:
data['Volatility 14'] = talib.ATR(data['high'],data['low'], data['close'], timeperiod=14)
data['Mid point price 14'] = talib.MIDPRICE(data['high'],data['low'], timeperiod=14)
data['vola 14 %']=data['Volatility 14']/data['Mid point price 14']*100
data['Rate of Change %'] = talib.ROC(data['close'],1)
data['RSI_14']= talib.RSI(data['close'],14) 
data['MOM'] = abs(abs(data['Rate of Change %'])-abs(data['vola 14 %']))
data['tail_up'] =(data['high']-data['close'])/data['close'] 
data['tail_down'] =(data['low']-data['close'])/data['close'] 
#data['MOM_14'] = talib.MOM(data['close'], timeperiod=14)

In [ ]:
final_stocks = total_tickers.merge(data,how='inner',left_on='ticker',right_on='ticker')
aapl_clusters = final_stocks[(final_stocks['date']==final_stocks['date'].max())].reset_index()

In [ ]:
patyh_excel = ''
aapl_clusters.to_excel(patyh_excel)

In [ ]:
aapl_clusters.head(2)

In [ ]:
# Расчет подскочивших и доступных в шорт

aapl_clusters[['ticker','Rate of Change %','vola 14 %','MOM']][(aapl_clusters['MOM']>=aapl_clusters['vola 14 %']) & (aapl_clusters['Short_available']=='Доступен') & (aapl_clusters['Rate of Change %']>0)].sort_values(by='MOM',ascending=False).reset_index().head(10)

In [ ]:
aapl_clusters[['ticker','Short_available','Rate of Change %','vola 14 %','MOM']][(aapl_clusters['Rate of Change %']>=aapl_clusters['MOM']) & (aapl_clusters['Short_available']=='Доступен')].sort_values(by='MOM',ascending=False).reset_index().head(10)

In [ ]:
# Расчет упавших

aapl_clusters[['ticker','Rate of Change %','vola 14 %','MOM']][(abs(aapl_clusters['Rate of Change %'])>=aapl_clusters['MOM']) & (aapl_clusters['Rate of Change %']<0)].sort_values(by='MOM',ascending=False).reset_index().head(10)

In [ ]:
sp = SectorPerformances(key='', output_format='pandas')
data, meta_data = sp.get_sector()

In [ ]:
sector = ''
data.to_excel(sector)

In [ ]:
ETF = ''

with open(ETF) as pt:
    total_ETF = pd.read_csv(pt,sep=';',encoding='utf8')

In [ ]:
tinkoff = {}
list_of_stocs=list(total_ETF['п»їETF'][0:60])
for ticker in list_of_stocs:
    tinkoff[ticker] = si.get_data(ticker, start_date='09-09-2021', index_as_date = False, interval='1d').sort_values(by='date')

ETF_DATA = pd.concat(tinkoff)
ETF_DATA.reset_index(drop=True, inplace=True)

In [ ]:
si.get_company_info('XSD')